# Liquidity Stress Tool

Interactive liquidity stress testing tool used to assess whether a fund can meet redemptions under market stress.

The model operates on aggregated liquidity buckets, mirroring how liquidity risk teams perform funding analysis.


In [9]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [10]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

from engine.assets import default_liquidity_profile
from engine.liquidity import run_waterfall, apply_stress
from engine.metrics import liquidity_metrics
from engine.assumptions import ASSUMPTIONS
from engine.scenario_translation import translate_scenario


In [11]:
# Helpers
def fmt_bn(x):
    return f"${x / 1e9:,.2f} bn"

def color_cash_used(val):
    if "$0.00" in val:
        return "color:#999"
    return "color:#1a7f37; font-weight:600"

def color_days(val):
    if val <= 1:
        return "color:#1a7f37; font-weight:600"
    elif val <= 30:
        return "color:#b26a00"
    else:
        return "color:#b42318; font-weight:600"

def render_summary(metrics, profile, scenario):
    status = "BREACH" if metrics["breach"] else "PASS"

    return widgets.HTML(f"""
    <div style="
        border:1px solid #ddd;
        border-radius:8px;
        padding:16px;
        margin-bottom:12px;
        max-width:650px;
        background:#fafafa;
    ">
        <h3>Stress Summary</h3>
        <ul>
            <li><b>Liquidity Status:</b> {status}</li>
            <li><b>Liquidity Coverage:</b> {metrics['liquidity_coverage']:.2f}x</li>
            <li><b>Days to Liquidity:</b> {metrics['days_to_liquidity']}</li>
            <li><b>Total Fund Value:</b> {fmt_bn(profile['market_value'].sum())}</li>
            <li><b>Scenario:</b>
                Equity {scenario['equity_drawdown']:.0%},
                Redemption {scenario['redemption']:.0%}
            </li>
        </ul>
    </div>
    """)


def plot_liquidity_charts(metrics, waterfall):
    fig, axes = plt.subplots(1, 2, figsize=(10, 3.5))

    axes[0].bar(
        ["Cash Required", "Cash Raised"],
        [metrics["cash_required"] / 1e9, metrics["cash_raised"] / 1e9,]
    )
    axes[0].set_title("Liquidity Coverage")
    axes[0].set_ylabel("CAD (bn)")

    wf = waterfall.sort_values("days_to_cash")
    axes[1].barh(wf["bucket"], wf["cash_used"] / 1e9)
    axes[1].set_title("Liquidity Waterfall")
    axes[1].set_xlabel("Cash Raised (CAD bn)")

    plt.tight_layout()
    plt.show()


In [57]:
equity_slider = widgets.FloatSlider(
    value = 0.0,
    min = 0.0,
    max = 0.99,
    step = 0.01,
    description = "Equity Drawdown",
    readout_show = True,
    readout_format = "0.0%",
    style = {"description_width": "150px"},
    layout = widgets.Layout(width = "280px")
)

redemption_slider = widgets.FloatSlider(
    value = 0.0,
    min = 0.0,
    max = 0.99,
    step = 0.01,
    description = "Fund Redemption",
    readout_show = True,
    readout_format = "0.0%",
    style = {"description_width": "150px"},
    layout = widgets.Layout(width = "280px")
)

freeze_cash = widgets.Checkbox(value=False, description="Freeze Cash & ST Bonds")
freeze_t1 = widgets.Checkbox(value=False, description="Freeze Public Equities [T+1]")
freeze_t5 = widgets.Checkbox(value=False, description="Freeze Public Credit [T+5]")
freeze_t30 = widgets.Checkbox(value=False, description="Freeze Real Estate [T+30]")
freeze_t90 = widgets.Checkbox(value=False, description="Freeze Infrastructure [T+90]")

controls = widgets.VBox(
    [widgets.HTML("<h3>Stress Inputs</h3>"), equity_slider, redemption_slider, freeze_cash, freeze_t1, freeze_t5, freeze_t30, freeze_t90,],
    layout = widgets.Layout(width = "360px", padding = "10px"),
)

In [12]:
output = widgets.Output()

In [13]:
def run_stress(_ = None):
    with output:
        clear_output()

        profile = default_liquidity_profile()

        if freeze_cash.value:
            profile.loc[profile["bucket"] == "Cash & Short-Term Bonds", "available"] = False
        if freeze_t1.value:
            profile.loc[profile["bucket"] == "Public Equities [T+1]", "available"] = False
        if freeze_t5.value:
            profile.loc[profile["bucket"] == "Public Credit [T+5]", "available"] = False
        if freeze_t30.value:
            profile.loc[profile["bucket"] == "Real Estate [T+30]", "available"] = False
        if freeze_t90.value:
            profile.loc[profile["bucket"] == "Infrastructure [T+90]", "available"] = False

        scenario = {
            "equity_drawdown": -equity_slider.value,
            "redemption": redemption_slider.value,
        }

        stressed_profile, liquidity_demand = translate_scenario(profile = profile, scenario = scenario, assumptions = ASSUMPTIONS,)
        stressed_profile = apply_stress(stressed_profile)
        waterfall, summary = run_waterfall(stressed_profile=stressed_profile, cash_required=liquidity_demand,)
        metrics = liquidity_metrics(summary)
        display(render_summary(metrics, profile, scenario))

        waterfall_display = waterfall.copy()
        for col in ["stressed_value", "cash_used", "remaining_value"]:
            waterfall_display[col] = waterfall_display[col].apply(fmt_bn)

        waterfall_display = waterfall_display.reset_index(drop=True)
        waterfall_display = waterfall_display.rename(columns={
            "bucket": "Asset Bucket",
            "stressed_value": "Stressed Value",
            "cash_used": "Cash Used",
            "remaining_value": "Remaining Value",
            "days_to_cash": "Days to Cash"
        })

        styled = waterfall_display.style \
            .set_properties(**{"text-align": "right"}) \
            .set_table_styles([
                {"selector": "th", "props": [("background", "#f2f2f2")]},
                {"selector": "th", "props": [("text-align", "right")]},
            ])

        display(widgets.HTML("""
        <h3 style="margin-bottom:8px;">
        Liquidity Waterfall (Post-Stress)
        </h3>
        <p style="color:#666; margin-top:0;">
        Order reflects liquidity speed from fastest to slowest.
        </p>
        """))

        styled = (
            waterfall_display.style
                .set_properties(**{"text-align": "right"})
                .set_properties(subset = ["Asset Bucket"], **{"text-align": "left"})
                .map(color_cash_used, subset = ["Cash Used"])
                .map(color_days, subset = ["Days to Cash"])
                .set_table_styles([
                    {"selector": "th", "props": [
                        ("background", "#f2f2f2"),
                        ("font-weight", "600"),
                        ("text-align", "right")
                    ]},
                    {"selector": "th:first-child", "props": [("text-align", "left")]},
                    {"selector": "td", "props": [("padding", "6px 10px")]},
                ])
        )


        display(styled)
        plot_liquidity_charts(metrics, waterfall)

In [14]:
for w in [equity_slider, redemption_slider, freeze_cash, freeze_t1, freeze_t5, freeze_t30, freeze_t90,]: w.observe(run_stress, names = "value")
app = widgets.HBox([controls, output])
display(app)
run_stress()